In [120]:
import datasets
import requests
import numpy as np

In [121]:
dataset = datasets.load_dataset('imdb')

train_dataset = dataset['train']
test_dataset = dataset['test']

url = "https://raw.githubusercontent.com/cjhutto/vaderSentiment/master/vaderSentiment/vader_lexicon.txt"
response = requests.get(url)
text = response.text
lines = text.split("\n")
values = [line.split() for line in lines]
dict = {}
for obj in values:
    if len(obj) != 13:
        continue
    dict.update({obj[0]: obj[1]})

Found cached dataset imdb (C:/Users/$0NT000-3G88GUGFR0B8/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 201.94it/s]


In [122]:
def lowercase_punctuation(text):
    newtext = text.lower()
    # remove <br /> tags
    newtext = newtext.replace('<br />', '')
    # remove punctuation
    newtext = ''.join([c if c not in "\"#$%&()*+,./:;<=>?@[\]^_`{|}~" else ' ' for c in newtext])
    return newtext.split()

In [123]:
def count_pronouns(word_list):
    count = 0
    for word in word_list:
        if word in ["i", "you", "we"]:
            count += 1
    return count

In [124]:
def lexicon_word_counter(word_list):
    positive_count = 0
    negative_count = 0
    lower_lexicon_value = -1
    upper_lexicon_value = 1
    for word in word_list:
        if word in dict.keys():
            sentimental_value = dict[word]
            if float(sentimental_value) < lower_lexicon_value:
                negative_count += 1
            elif float(sentimental_value) > upper_lexicon_value:
                positive_count += 1
    return positive_count, negative_count

In [125]:
def vector_generator(dataset):
    resulting_vector = []
    for obj in dataset:
        word_list = lowercase_punctuation(obj['text'])
        check_no = 1 if word_list.__contains__('no') else 0
        nb_pronouns = count_pronouns(word_list)
        check_exclamation = 1 if word_list.__contains__('!') else 0
        log_nb_words = np.log(len(word_list))
        positive_count, negative_count = lexicon_word_counter(word_list)
        resulting_vector.append([check_no, nb_pronouns, check_exclamation, log_nb_words, positive_count, negative_count])
    return resulting_vector

In [ ]:
train_result = vector_generator(train_dataset)